<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 26th, 2021</h5>

# Set up and requirements

### Import Python packages

In [1]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.t12_utils as t12
from utils.zooniverse_utils import retrieve_zoo_info, populate_subjects
import getpass

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Initiate SQL database and populate sites, movies and species

In [2]:
# Specify the path of the movies 
movies_path = "/uploads"

# Specify the path of the sql database
db_path = "koster_lab.db"

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --movies_path $movies_path --db_path $db_path

Updated sites
Updated movies
Updated species


### Retrieve Zooniverse information

In [3]:
# Save your user name, password and Zooniverse project number.
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')
project_n = getpass.getpass('Enter the number of the Zooniverse project')

Enter your Zooniverse user········
Enter your Zooniverse password········
Enter the number of the Zooniverse project········


In [4]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects", "workflows", "classifications"]

# Retrieve and store the Zooniverse information required throughout the tutorial in a dictionary
zoo_info_dict = retrieve_zoo_info(zoo_user, zoo_pass, project_n, zoo_info)

Connecting to the Zooniverse project
Retrieving subjects from Zooniverse
subjects were retrieved succesfully
Retrieving workflows from Zooniverse
workflows were retrieved succesfully
Retrieving classifications from Zooniverse
classifications were retrieved succesfully


In [5]:
# Populate the sql with subjects already uploaded to Zooniverse
populate_subjects(zoo_info_dict["subjects"], project_n, db_path)

Updated subjects


### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

In [7]:
workflows_df = zoo_info_dict["workflows"]

In [18]:
sp = workflows_df.display_name.unique().tolist()[0]

In [19]:
workflows_df[workflows_df.display_name==sp].workflow_id.unique()[0]

11767

In [ ]:
#Combine the workflow ID and title
workflows_df["workflow_id_title"] = workflows_df.display_name + " (" + workflows_df.workflow_id.apply(str) + ")"

In [ ]:
index_1 = workflows_df.workflow_id_title.unique().tolist().index("Species identification (11767)")
index_1
workflows_df.workflow_id.unique().tolist()[index_1]

In [ ]:
workflows_df.workflow_id.unique().tolist()

In [6]:
# Display a selectable list of workflow names and a list of versions of the workflow of interest
workflow_id, workflow_version, subj_type = t12.choose_workflows(zoo_info_dict["workflows"])

Dropdown(description='Workflow id:', options=('Species identification', 'test ', 'Species location (intermedia…

Dropdown(description='Minimum workflow version:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, …

Dropdown(description='Subject type:', index=1, options=('frame', 'clip'), value='clip')

IndexError: list index out of range

In [ ]:
# Retrieve classifications from the workflow of interest
total_df, class_df = t12.get_classifications(workflow_id.value,
                                             workflow_version.value, 
                                             subj_type.value, 
                                             zoo_info_dict["classifications"], 
                                             zoo_info_dict["subjects"])

### Step 2: Aggregate classifications received on the workflow of interest

In [ ]:
# Specify the agreement threshold required among cit scientists
agg_users, min_users = t12.choose_agg_parameters()

In [ ]:
agg_class_df = t12.aggregrate_classifications(class_df, subj_type.value, agg_users.value, min_users.value)

### Step 3: Summarise the number of classifications based on the agreement specified

In [ ]:
agg_class_df.groupby("label").agg({"subject_ids": "count"})

### Step 4: Display the aggregated classifications in a table

In [ ]:
# Display the dataframe into a table
agg_class_df

### Step 5: Use the subject explorer widget to get more information about specific subjects and their classifications

In [ ]:
# Get all classified subjects from specified workflows
subject_df = t12.process_clips(class_df) if subj_type.value == "clip" else t13.process_frames(class_df)

t12.launch_viewer(total_df, subject_df)

In [ ]:
# END